In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish_validation_train


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

# from src.ad_hmm import sku_predict
# from src.ad_stl_model import stl_model
from src.utils import *

In [3]:
%load_ext autoreload 
%autoreload 2

### y-actual

In [4]:
original_validation = pd.read_excel("../../data/Validation_Data.xlsx").drop_duplicates()
y_act = original_validation[["Encoded_SKU_ID","SALES_DATE","DAILY_UNITS"]].rename(columns = {"DAILY_UNITS":"actual"})
y_act.head()

,Encoded_SKU_ID,SALES_DATE,actual
0,23,2022-08-06,3
1,100,2022-08-02,2
2,121,2022-08-06,6
3,207,2022-08-03,0
4,416,2022-08-05,9


###  Read all predictions

In [5]:
#y_null
y_pred_null = pd.read_csv("../../results/null_model_results_final.csv",index_col=0)
y_pred_null["SALES_DATE"] = pd.to_datetime(y_pred_null["SALES_DATE"])
y_pred_null.Encoded_SKU_ID.nunique()

539

In [6]:
#y_holt
y_pred_holt = pd.read_csv("../../results/holt_rst_final.csv",index_col=0)
y_pred_holt["SALES_DATE"] = pd.to_datetime(y_pred_holt["SALES_DATE"])
y_pred_holt.Encoded_SKU_ID.nunique()

25

In [7]:
#y_hmm
y_pred_hmm = pd.read_csv("../../results/hmm_result_final_all.csv").drop(columns=["Unnamed: 0"])
y_pred_hmm["SALES_DATE"] = pd.to_datetime(y_pred_hmm["SALES_DATE"])
y_pred_hmm.Encoded_SKU_ID.nunique()

204

In [8]:
#y_prophet_stl
import pickle
with open('../../data/stl_prophet_final_results_127sku.pkl', 'rb') as f:
    all_preds = pickle.load(f)
    
y_pred_prophet_stl_mstl = pd.DataFrame()
for i in range(len(all_preds)):
    if all_preds[i]!= None:
        y_pred_prophet_stl_mstl = pd.concat([y_pred_prophet_stl_mstl,all_preds[i][1]])
y_pred_prophet_stl_mstl = y_pred_prophet_stl_mstl.reset_index().drop(columns=["actual"])
y_pred_prophet_stl_mstl["SALES_DATE"] = pd.to_datetime(y_pred_prophet_stl_mstl["SALES_DATE"])
y_pred_prophet_stl_mstl.Encoded_SKU_ID.nunique()

Importing plotly failed. Interactive plots will not work.


102

In [9]:
y_pred_stl = y_pred_prophet_stl_mstl[["SALES_DATE","STL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"STL_prediction":"predicted"})

y_pred_mstl = y_pred_prophet_stl_mstl[["SALES_DATE","MSTL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"MSTL_prediction":"predicted"})

y_pred_prophet = y_pred_prophet_stl_mstl[["SALES_DATE","prophet_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"prophet_prediction":"predicted"})

### Merge all predictions

In [10]:
print("null",y_pred_null["Encoded_SKU_ID"].nunique())
print("stl",y_pred_stl["Encoded_SKU_ID"].nunique())
print("mstl",y_pred_mstl["Encoded_SKU_ID"].nunique())
print("holt",y_pred_holt["Encoded_SKU_ID"].nunique())
print("hmm",y_pred_hmm["Encoded_SKU_ID"].nunique())
print("prophet",y_pred_prophet["Encoded_SKU_ID"].nunique())

null 539
stl 102
mstl 102
holt 25
hmm 204
prophet 102


In [11]:
y_pred_holt1 = y_pred_holt.rename(columns={"predicted":"predicted_holt"})

In [12]:
all_predictions = y_pred_hmm.merge(y_pred_prophet_stl_mstl,on=["Encoded_SKU_ID","SALES_DATE"],how="outer"\
                                            ).merge(y_pred_null,on=["Encoded_SKU_ID","SALES_DATE"],how="outer",\
                                                   suffixes = ("_15","_null"),\
                                                   ).merge(y_pred_holt1,on=["Encoded_SKU_ID","SALES_DATE"],how="outer"\
                                                          ,suffixes=("","_holt"))
all_predictions.head()

,SALES_DATE,predicted_15,predicted_30,predicted_45,predicted_60,predicted_90,predicted_max,Encoded_SKU_ID,STL_prediction,MSTL_prediction,prophet_prediction,predicted_null,predicted_holt
0,2022-08-01,3.00,3.00,3.00,3.00,2.00,3.00,1,NaN,NaN,NaN,1,NaN
1,2022-08-01,1.55,0.45,0.45,0.45,2.00,1.00,1,NaN,NaN,NaN,1,NaN
2,2022-08-01,1.55,1.55,1.00,2.00,2.00,2.00,1,NaN,NaN,NaN,1,NaN
3,2022-08-01,3.00,3.00,2.00,3.00,1.45,0.45,1,NaN,NaN,NaN,1,NaN
4,2022-08-01,0.55,0.55,1.00,1.00,0.45,2.00,1,NaN,NaN,NaN,1,NaN


In [13]:
all_predictions = all_predictions.rename(columns={
    "STL_prediction":"predicted_STL",\
    "MSTL_prediction":"predicted_MSTL",\
    "prophet_prediction":"predicted_prophet"
})

### Create Aggregare prediction

In [36]:
rmse_all_preds = pd.DataFrame()
first_flag = True
for col in all_predictions.columns:
    if "pred" in col:
        y_pred_i = all_predictions[["Encoded_SKU_ID","SALES_DATE",col\
                                   ]].rename(columns={col:"predicted"})
        y_pred_i["SALES_DATE"] = pd.to_datetime(y_pred_i["SALES_DATE"])
        rmse_pred_i = rmse_sku(y_act,y_pred_i)
        rmse_pred_i.columns = [f"{col_rmse}_{col.split('_')[-1]}" for col_rmse in rmse_pred_i.columns]
        if first_flag:
            rmse_all_preds = rmse_pred_i
            print (col,rmse_all_preds.shape)
            first_flag = False
        else:
            rmse_all_preds = rmse_all_preds.merge(rmse_pred_i, left_index = True, right_index = True,\
                                suffixes = ("",f"_{col.split('_')[-1]}"))
            print (col,rmse_all_preds.shape)

predicted_15 (539, 3)
predicted_30 (539, 6)
predicted_45 (539, 9)
predicted_60 (539, 12)
predicted_90 (539, 15)
predicted_max (539, 18)
predicted_STL (539, 21)
predicted_MSTL (539, 24)
predicted_prophet (539, 27)
predicted_null (539, 30)
predicted_holt (539, 33)


In [37]:
rmse_cols = [col for col in rmse_all_preds.columns if "rmse_du" in col]
rmse_cols

['rmse_du_15',
 'rmse_du_30',
 'rmse_du_45',
 'rmse_du_60',
 'rmse_du_90',
 'rmse_du_max',
 'rmse_du_STL',
 'rmse_du_MSTL',
 'rmse_du_prophet',
 'rmse_du_null',
 'rmse_du_holt']

In [57]:
rmse_all_preds["min_rmse"] = rmse_all_preds[rmse_cols].min(axis=1,skipna=True)

rmse_all_preds["argmin_period"] = np.nanargmin(rmse_all_preds[rmse_cols].values,axis=1)

rmse_all_preds["min_period"] = rmse_all_preds["argmin_period"].apply(lambda x: rmse_cols[x])

In [58]:
rmse_all_preds["min_period"].value_counts()

rmse_du_null       326
rmse_du_prophet     40
rmse_du_STL         33
rmse_du_15          30
rmse_du_60          23
rmse_du_max         20
rmse_du_90          17
rmse_du_30          16
rmse_du_45          16
rmse_du_holt         9
rmse_du_MSTL         9
Name: min_period, dtype: int64

In [59]:
period_col_map= {
    "rmse_du_15":"predicted_15", "rmse_du_30":"predicted_30", "rmse_du_45":"predicted_45",\
    "rmse_du_60":"predicted_60", "rmse_du_90":"predicted_90", "rmse_du_max":"predicted_max",\
    "rmse_du_STL":"predicted_STL","rmse_du_MSTL":"predicted_MSTL","rmse_du_null":"predicted_null",\
    "rmse_du_holt":"predicted_holt","rmse_du_prophet":"predicted_prophet"
}

In [60]:
y_pred_best = pd.DataFrame()
cnt = 0
for sku_id,best_period in zip(rmse_all_preds.index,rmse_all_preds["min_period"]):
    best_col = period_col_map[best_period]
    mask_sku = all_predictions.Encoded_SKU_ID==sku_id
    y_pred_best_sku = all_predictions.loc[mask_sku][["Encoded_SKU_ID","SALES_DATE",best_col\
                                                                  ]].rename(columns={best_col:"predicted"})
    y_pred_best = pd.concat([y_pred_best,y_pred_best_sku])

In [61]:
#QC
y_act.Encoded_SKU_ID.nunique(),y_pred_null.Encoded_SKU_ID.nunique(),y_pred_best.Encoded_SKU_ID.nunique()

(539, 539, 539)

In [62]:
rmse_val_null,rmse_null_best = rmse(y_act,y_pred_null),rmse(y_act,y_pred_best)
rmse_val_null,rmse_null_best

(5.290976671399268, 5.087027787731684)

In [65]:
rmse_best_sku = rmse_sku(y_act,y_pred_best)

In [66]:
rmse_best_sku.head()

,rmse_du,mean_du,pct_rmse
Encoded_SKU_ID,,,
469,91.411940,396.000000,0.230838
430,40.362642,94.714286,0.426152
50,32.671525,73.857143,0.442361
557,22.696759,33.285714,0.681877
271,16.694738,28.428571,0.587252


In [67]:
rmse_best_sku.tail()

,rmse_du,mean_du,pct_rmse
Encoded_SKU_ID,,,
215,0.316228,0.100000,3.162278
407,0.000000,0.142857,0.000000
228,0.000000,0.111111,0.000000
542,0.000000,0.000000,NaN
524,0.000000,0.142857,0.000000


### RMSE

In [14]:
rmse_all_preds = pd.DataFrame()
first_flag = True
for col in all_predictions.columns:
    if "pred" in col:
        y_pred_i = all_predictions[["Encoded_SKU_ID","SALES_DATE",col\
                                   ]].rename(columns={col:"predicted"})
        y_pred_i["SALES_DATE"] = pd.to_datetime(y_pred_i["SALES_DATE"])
        rmse_pred_i = rmse_sku(y_act,y_pred_i)
        rmse_pred_i.columns = [f"{col_rmse}_{col.split('_')[-1]}" for col_rmse in rmse_pred_i.columns]
        if first_flag:
            rmse_all_preds = rmse_pred_i
            print (col,rmse_all_preds.shape)
            first_flag = False
        else:
            rmse_all_preds = rmse_all_preds.merge(rmse_pred_i, left_index = True, right_index = True,\
                                suffixes = ("",f"_{col.split('_')[-1]}"))
            print (col,rmse_all_preds.shape)

predicted_15 (539, 3)
predicted_30 (539, 6)
predicted_45 (539, 9)
predicted_60 (539, 12)
predicted_90 (539, 15)
predicted_max (539, 18)
predicted_STL (539, 21)
predicted_MSTL (539, 24)
predicted_prophet (539, 27)
predicted_null (539, 30)
predicted_holt (539, 33)


In [15]:
rmse_all_preds

,rmse_du_15,mean_du_15,pct_rmse_15,rmse_du_30,mean_du_30,pct_rmse_30,rmse_du_45,mean_du_45,pct_rmse_45,rmse_du_60,...,pct_rmse_MSTL,rmse_du_prophet,mean_du_prophet,pct_rmse_prophet,rmse_du_null,mean_du_null,pct_rmse_null,rmse_du_holt,mean_du_holt,pct_rmse_holt
Encoded_SKU_ID,,,,,,,,,,,,,,,,,,,,,
482,5.644213,6.285714,0.897943,4.894640,6.285714,0.778693,6.126086,6.285714,0.974605,4.765988,...,NaN,NaN,6.285714,NaN,7.819390,6.285714,1.243994,NaN,6.285714,NaN
315,4.813708,5.000000,0.962742,4.988594,5.000000,0.997719,4.521457,5.000000,0.904291,3.184785,...,NaN,NaN,5.000000,NaN,3.605551,5.000000,0.721110,NaN,5.000000,NaN
183,4.519798,5.714286,0.790965,4.292851,5.714286,0.751249,2.688800,5.714286,0.470540,2.484308,...,NaN,NaN,5.714286,NaN,2.563480,5.714286,0.448609,NaN,5.714286,NaN
319,4.292851,3.142857,1.365907,2.780095,3.142857,0.884576,3.433085,3.142857,1.092345,2.535674,...,NaN,NaN,3.142857,NaN,2.420153,3.142857,0.770049,NaN,3.142857,NaN
312,4.017817,5.285714,0.760128,3.817254,5.285714,0.722183,5.554921,5.285714,1.050931,5.190685,...,NaN,NaN,5.285714,NaN,4.358899,5.285714,0.824657,NaN,5.285714,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,NaN,6.428571,NaN,NaN,6.428571,NaN,NaN,6.428571,NaN,NaN,...,NaN,NaN,6.428571,NaN,2.725541,6.428571,0.423973,NaN,6.428571,NaN
567,NaN,3.142857,NaN,NaN,3.142857,NaN,NaN,3.142857,NaN,NaN,...,0.836491,1.522228,3.142857,0.484345,1.511858,3.142857,0.481046,NaN,3.142857,NaN
568,NaN,3.428571,NaN,NaN,3.428571,NaN,NaN,3.428571,NaN,NaN,...,0.530873,2.000404,3.428571,0.583451,2.329929,3.428571,0.679563,NaN,3.428571,NaN


In [16]:
rmse_cols = [col for col in rmse_all_preds.columns if "rmse_du" in col]
rmse_cols

['rmse_du_15',
 'rmse_du_30',
 'rmse_du_45',
 'rmse_du_60',
 'rmse_du_90',
 'rmse_du_max',
 'rmse_du_STL',
 'rmse_du_MSTL',
 'rmse_du_prophet',
 'rmse_du_null',
 'rmse_du_holt']

In [17]:
# #mute rmse null wherever other model is available to prevent data leak
# rmse_cols_wo_null = [x for x in rmse_cols if "null" not in x]
# atleast_one_model_not_nan = (~rmse_all_preds[rmse_cols_wo_null].isna()).sum(axis=1)>0
# atleast_one_model_not_nan.sum(),(~atleast_one_model_not_nan).sum()

In [18]:
# rmse_all_preds.loc[atleast_one_model_not_nan,"rmse_null"]=np.nan

### Create y_best cumulatively | Ensemble*

In [19]:
def create_y_best(rmse_all_preds,step_cols,period_col_map,step=""):
    rmse_all_preds[f"min_rmse_{step}"] = rmse_all_preds[step_cols].min(axis=1,skipna=True)
    rmse_all_preds[f"argmin_period_{step}"] = np.argmin(rmse_all_preds[step_cols].values,axis=1)
    rmse_all_preds[f"min_period_{step}"] = rmse_all_preds[f"argmin_period_{step}"].apply(lambda x: step_cols[x])
    y_pred_best = pd.DataFrame()
    cnt = 0
    for sku_id,best_period in zip(rmse_all_preds.index,rmse_all_preds[f"min_period_{step}"]):
        best_col = period_col_map[best_period]
        mask_sku = all_predictions.Encoded_SKU_ID==sku_id
        y_pred_best_sku = all_predictions.loc[mask_sku][["Encoded_SKU_ID","SALES_DATE",best_col\
                                                                  ]].rename(columns={best_col:"predicted"})
        y_pred_best = pd.concat([y_pred_best,y_pred_best_sku])
    return (y_pred_best)

In [20]:
period_col_map= {
    "rmse_du_15":"predicted_15", "rmse_du_30":"predicted_30", "rmse_du_45":"predicted_45",\
    "rmse_du_60":"predicted_60", "rmse_du_90":"predicted_90", "rmse_du_max":"predicted_max",\
    "rmse_du_STL":"predicted_STL","rmse_du_MSTL":"predicted_MSTL","rmse_du_null":"predicted_null",\
    "rmse_du_holt":"predicted_holt","rmse_du_prophet":"predicted_prophet"
}

### Step 1: Null + Holt + Prophet

In [21]:
step = "2"
step_cols = ['rmse_du_STL',
 'rmse_du_MSTL',
 'rmse_du_prophet',
 'rmse_du_null',
 'rmse_du_holt']

y_best1 = create_y_best(rmse_all_preds,step_cols,period_col_map,step)
assert y_best1.Encoded_SKU_ID.nunique()==539, "y_best does not have all SKUs"
rmse_val_null,rmse_null_best = rmse(y_act,y_pred_null),rmse(y_act,y_best1)
rmse_val_null,rmse_null_best,((rmse_val_null-rmse_null_best)*100)/rmse_null_best

(5.290976671399268, 1.0592605892214235, 399.49717050156994)

In [22]:
rmse_all_preds[f"min_period_{step}"].value_counts()

rmse_du_STL     439
rmse_du_holt     97
rmse_du_null      3
Name: min_period_2, dtype: int64

In [23]:
### Step 1: Null + Holt + Prophet

step = "3"
step_cols = rmse_cols

y_best1 = create_y_best(rmse_all_preds,step_cols,period_col_map,step)
assert y_best1.Encoded_SKU_ID.nunique()==539, "y_best does not have all SKUs"
rmse_val_null,rmse_null_best = rmse(y_act,y_pred_null),rmse(y_act,y_best1)
rmse_val_null,rmse_null_best,((rmse_val_null-rmse_null_best)*100)/rmse_null_best

(5.290976671399268, 1.2366911080659224, 327.8333236885561)

In [26]:
rmse_all_preds[f"min_period_{step}"].value_counts()

rmse_du_15      384
rmse_du_STL     153
rmse_du_holt      1
rmse_du_60        1
Name: min_period_3, dtype: int64